In [2]:
from collections import deque, defaultdict
from UserClasses import generateTree, generateList, ListNode, TreeNode
import math
import functools
import re
import heapq
import bisect 

In [64]:
#2563. Count the Number of Fair Pairs
class CountFairPairs:
    def _searchLower(self, nums, target, start_index):
        left_index, right_index = start_index, len(nums) - 1
        if target > nums[right_index]:
            return None
        if target < nums[left_index]:
            return left_index
        while right_index - left_index > 1:
            middle_index = (left_index + right_index) // 2
            if nums[middle_index] >= target:
                right_index = middle_index
            else:
                left_index = middle_index
        return left_index if nums[left_index] == target else right_index
        
    def _searchUpper(self, nums, target, start_index):
        left_index, right_index = start_index, len(nums) - 1
        if target > nums[right_index]:
            return right_index
        if target < nums[left_index]:
            return None
        while right_index - left_index > 1:
            middle_index = (left_index + right_index) // 2
            if nums[middle_index] > target:
                right_index = middle_index
            else:
                left_index = middle_index
        return right_index if nums[right_index] == target else left_index
        
    def countFairPairs(self, nums, lower, upper):
        nums.sort()
        result = 0
        for index, num in enumerate(nums[:-1]):
            if num + nums[-1] < lower:
                continue
            if num + nums[index+1] > upper:
                break
            lower_index = self._searchLower(nums, lower - num, index + 1)
            upper_index = self._searchUpper(nums, upper - num, index + 1)
            if ((lower_index is not None and upper_index is not None)
                and (lower_index <= upper_index)):
                result += upper_index - lower_index + 1
        return result

In [70]:
#1574. Shortest Subarray to be Removed to Make Array Sorted
class FindLengthOfShortestSubarray:
    def _searchEqualOrLess(self, arr, cur_index):
        for index in range(cur_index - 1, -1, -1):
            if arr[index] <= arr[cur_index]:
                return index
        return -1

    def _searchEqualOrGreater(self, arr, cur_index):
        for index in range(cur_index + 1, len(arr)):
            if arr[index] >= arr[cur_index]:
                return index
        return len(arr)
    
    def _checkNonDecreasing(self, arr):
        for index in range(1, len(arr)):
            if arr[index-1] > arr[index]:
                return False
        return True

    def _findLargestSubarray(self, arr, repeat_flag):
        result = 1
        for index in range(1, len(arr)):
            if arr[index-1] > arr[index]:
                if not repeat_flag:
                    return result
                result = 0
            result += 1
        return result
        
    def findLengthOfShortestSubarray(self, arr):
        decreasing_flag = False
        right_target_index = None
        for index in range(1, len(arr)):
            if arr[index-1] > arr[index]:
                if not decreasing_flag:
                    right_target_index = index - 1
                decreasing_flag = True
            elif decreasing_flag:
                left_index = self._searchEqualOrLess(arr, index - 1)
                right_index = self._searchEqualOrGreater(arr, right_target_index)
                if self._checkNonDecreasing(arr[index-1:]):
                    print(index - 1, left_index)
                    print(right_target_index, right_index)
                    return min(index - left_index - 2, right_index - right_target_index - 1)
                break
        return min(len(arr) - self._findLargestSubarray(arr, True),
                   len(arr) - self._findLargestSubarray(arr, False))

In [6]:
#1455. Check If a Word Occurs As a Prefix of Any Word in a Sentence
class IsPrefixOfWord(object):
    def isPrefixOfWord(self, sentence, searchWord):
        string, index = '', 1
        for char in sentence:
            if char == ' ':
                if string[:len(searchWord)] == searchWord:
                    return index
                string = ''
                index += 1
            else:
                string += char
        if string[:len(searchWord)] == searchWord:
            return index
        return -1

In [123]:
#2097. Valid Arrangement of Pairs
class ValidArrangement(object):
    def _buildPath(self, begin_node, graph):
        result = []
        stack = deque([begin_node])
        while stack:
            cur_node = stack[-1]
            if cur_node in graph:
                stack.append(graph[cur_node].pop())
                if not graph[cur_node]:
                    graph.pop(cur_node)
            else:
                result.append(stack.pop())
        result.reverse()
        return result    

    def _buildResult(self, path):
        result = []
        for index in range(1, len(path)):
            result.append([path[index-1], path[index]])
        return result
    
    def validArrangement(self, pairs):
        graph, node_degree = {}, {}
        for begin, end in pairs:
            graph.setdefault(begin, set()).add(end)
            node_degree[begin] = node_degree.setdefault(begin, 0) + 1
            node_degree[end] = node_degree.setdefault(end, 0) - 1

        begin_node = pairs[0][0]
        for node, degree in node_degree.items():
            if degree == 1:
                begin_node = node
                break
            
        path = self._buildPath(begin_node, graph)
        return self._buildResult(path)

In [1]:
#2109. Adding Spaces to a String
class AddSpaces:
    def addSpaces(self, s, spaces):
        res_string_arr = []
        start_index = 0
        spaces.append(len(s))
        for end_index in spaces:
            res_string_arr.append(s[start_index:end_index])
            start_index = end_index
        return ' '.join(res_string_arr)

In [57]:
#3243. Shortest Distance After Road Addition Queries I
class ShortestDistanceAfterQueries:
    def _changeGraph(self, graph, cur_node, prev_node, node_distance):
        if node_distance[cur_node] > node_distance[prev_node] + 1:
            node_distance[cur_node] = node_distance[prev_node] + 1
            if cur_node in graph:
                for next_node in graph[cur_node]:
                    self._changeGraph(graph, next_node, cur_node, node_distance)
            
    def shortestDistanceAfterQueries(self, n, queries):
        graph = {node: {node+1} for node in range(n-1)}
        node_distance = {node: node for node in range(n)}
        result = []
        for start_node, end_node in queries:
            graph[start_node].add(end_node)
            self._changeGraph(graph, end_node, start_node, node_distance)
            result.append(node_distance[n-1])
        return result

In [75]:
class СanMakeSubsequence:
    @functools.cache
    def _getPrevLetter(self, letter):
        return 'z' if letter == 'a' else chr(ord(letter)-1)
        
    def canMakeSubsequence(self, str1, str2):
        str2_index = 0
        for char in str1:
            if ((char == str2[str2_index])
                or (char == self._getPrevLetter(str2[str2_index]))):
                str2_index += 1
            if str2_index == len(str2):
                return True
        return False

In [41]:
#2337. Move Pieces to Obtain a String
class CanChange:  
    def _fillArr(self, arr, string, letter):
        char_amount = 0
        for index, char in enumerate(string):
            char_amount += char == letter
            arr[index][letter] = char_amount
        
    def canChange(self, start, target):
        start_seq = [char for char in start if char!='_']
        target_seq = [char for char in target if char!='_']
        if ''.join(start_seq) != ''.join(target_seq):
            return False
        
        start_index_amount = [{'L': None, 'R': None} for _ in range(len(start))]
        self._fillArr(start_index_amount, start, 'R')
        self._fillArr(start_index_amount[::-1], start[::-1], 'L')
        target_index_amount = [{'L': None, 'R': None} for _ in range(len(target))]
        self._fillArr(target_index_amount, target, 'R')
        self._fillArr(target_index_amount[::-1], target[::-1], 'L')
        for index in range(len(start_index_amount)):
            if ((target_index_amount[index]['L'] > start_index_amount[index]['L'])
                or (target_index_amount[index]['R'] > start_index_amount[index]['R'])):
                return False
        return True  

In [ ]:
#2554. Maximum Number of Integers to Choose From a Range I
class MaxCount:
    def maxCount(self, banned, n, maxSum):
        exclude_nums = set(banned)
        result, res_sum = 0, 0
        for num in range(1, n+1):
            if num not in exclude_nums:
                res_sum += num
                if res_sum > maxSum:
                    break
                result += 1
        return result

In [32]:
#2577. Minimum Time to Visit a Cell In a Grid
class MinimumTime:
    def _getNextTime(self, grid, next_node, cur_time):
        target_time = grid[next_node[0]][next_node[1]] - 1
        result = target_time + (target_time - cur_time) % 2 if cur_time < target_time else cur_time
        return result + 1 
        
    def minimumTime(self, grid):
        if (grid[0][1] > 1) and (grid[1][0] > 1):
            return -1
        rows, cols = len(grid), len(grid[0])
        stack = deque([(0, (0, 0))])
        checked_nodes = {(0, 0)}
        while stack:
            cur_time, cur_node = stack.popleft()
            if cur_node == (rows-1, cols-1):
                return cur_time
            for row_inc in range(-1, 2):
                for col_inc in range(-1, 2):
                    if abs(row_inc+col_inc) == 1:
                        next_node = (cur_node[0] + row_inc, cur_node[1] + col_inc)
                        if ((0 <= next_node[0] < rows) 
                            and (0 <= next_node[1] < cols) 
                            and (next_node not in checked_nodes)):
                            next_time = self._getNextTime(grid, next_node, cur_time)
                            bisect.insort(stack, (next_time, next_node))
                            checked_nodes.add(next_node)

In [10]:
#3152. Special Array II
class IsArraySpecial:
    def isArraySpecial(self, nums, queries):
        special_prefix = [0]
        for index, num in enumerate(nums[1:], 1):
            special_prefix.append(special_prefix[-1]+1-(num+nums[index-1])%2)
        result = []
        for begin, end in queries:
            result.append((special_prefix[end]-special_prefix[begin])==0)
        return result

In [10]:
#2981. Find Longest Special Substring That Occurs Thrice I
class MaximumLength:
    def _updateAmount(self, amount, substring):
        for degree in range(1, substring[1]+1):
            new_substring = substring[0] * degree
            amount[new_substring] = amount.setdefault(new_substring, 0) \
                                + substring[1] + 1 - degree
            
    def maximumLength(self, s):
        amount = {}
        prev_substring = [s[0], 1]
        for iter, char in enumerate(s[1:], 1):
            if char == prev_substring[0]:
                prev_substring[1] += 1
            else:
                self._updateAmount(amount, prev_substring)
                prev_substring = [char, 1]
        self._updateAmount(amount, prev_substring)
        result = -1
        for substring, amount in amount.items():
            if amount >= 3:
                result = max(result, len(substring))
        return result

In [31]:
#2779. Maximum Beauty of an Array After Applying Operation
class MaximumBeauty:
    def _searchLimitIndex(self, arr, target):
        left_index, right_index = 0, len(arr) - 1
        if target > arr[-1]:
            return right_index
        while right_index - left_index > 1:
            middle_index = (left_index + right_index) // 2
            if arr[middle_index] <= target:
                left_index = middle_index
            else: 
                right_index = middle_index
        return right_index if arr[right_index] == target else left_index
        
    def maximumBeauty(self, nums, k):
        nums.sort()
        result = 1
        for left_limit, num in enumerate(nums[:-1]):
            target = num + 2 * k
            if ((left_limit + result < len(nums)) 
                and (nums[left_limit+result] <= target)):
                right_limit = left_limit + self._searchLimitIndex(nums[left_limit:], target)
                result = max(result, right_limit - left_limit + 1)
        return result

In [5]:
#2558. Take Gifts From the Richest Pile
class PickGifts:
    def pickGifts(self, gifts, k):
        sorted_gifts = deque(sorted(gifts))
        for _ in range(k):
            max_val = sorted_gifts.pop()
            bisect.insort(sorted_gifts, int(math.sqrt(max_val)))
        return sum(sorted_gifts)

In [11]:
#2290. Minimum Obstacle Removal to Reach Corner
class MinimumObstacles:
    def minimumObstacles(self, grid):
        rows, cols = len(grid), len(grid[0])
        stack = deque([(0, (0, 0))])
        checked_cells = {(0, 0)}
        while stack:
            cur_val, cur_cell = stack.popleft()
            if cur_cell == (rows-1, cols-1):
                return cur_val
            for row_diff in range(-1, 2):
                for col_diff in range(-1, 2):
                    if abs(row_diff+col_diff) == 1:
                        next_cell = (cur_cell[0]+row_diff, cur_cell[1]+col_diff)
                        if ((0 <= next_cell[0] < rows)
                            and (0 <= next_cell[1] < cols)
                            and (next_cell not in checked_cells)):
                            next_val = cur_val if grid[next_cell[0]][next_cell[1]]==0 else cur_val+1
                            bisect.insort(stack, (next_val, next_cell))
                            checked_cells.add(next_cell)

In [15]:
#2593. Find Score of an Array After Marking All Elements
class FindScore:
    def findScore(self, nums):
        num_indexes = {}
        for index, num in enumerate(nums):
            num_indexes.setdefault(num, deque([])).append(index)
        sorted_nums = list(sorted(num_indexes.keys()))
        used_indexes = set()
        result = 0
        for num in sorted_nums:
            while num_indexes[num]:
                index = num_indexes[num].popleft()
                if index not in used_indexes:
                    for used_index in range(index-1, index+2):
                        if 0 <= used_index < len(nums):
                            used_indexes.add(used_index)
                    result += num
        return result

In [46]:
#2054. Two Best Non-Overlapping Events
class MaxTwoEvents:
    def _searchStart(self, postfix, target):
        left_index, right_index = 0, len(postfix) - 1
        if target > postfix[right_index][0]:
            return 0
        while right_index - left_index > 1:
            if postfix[left_index] == target:
                return postfix[left_index][1]
            if postfix[right_index] == target:
                return postfix[right_index][1]
            middle_index = (left_index + right_index) // 2
            if postfix[middle_index][0] < target:
                left_index = middle_index
            else:
                right_index = middle_index
        return postfix[right_index][1]
        
    def maxTwoEvents(self, events):
        start_value = {}
        for start, end, value in events:
            start_value[start] = max(start_value.setdefault(start, 0), value)
        postfix = deque([])
        for start in sorted(start_value.keys(), reverse=True):
            if not postfix:
                postfix.appendleft((start, start_value[start]))
            else:
                postfix.appendleft((start, max(postfix[0][1], start_value[start])))
        result = postfix[0][1]
        for start, end, value in events:
            full_value = value + self._searchStart(postfix, end+1)
            result = max(result, full_value)
        return result

In [13]:
#3264. Final Array State After K Multiplication Operations I
class GetFinalState:
    def getFinalState(self, nums, k, multiplier):
        nums_index = deque(sorted([(num, index) for index, num in enumerate(nums)]))
        for _ in range(k):
            cur_num, cur_index = nums_index.popleft()
            bisect.insort(nums_index, (cur_num*multiplier, cur_index))
        result = [None for _ in range(len(nums_index))]
        for num, index in nums_index:
            result[index] = num
        return result

In [41]:
#2182. Construct String With Repeat Limit
class RepeatLimitedString:
    def repeatLimitedString(self, s, repeatLimit):
        letters = list(s)
        letters.sort(key=ord, reverse=True)
        replace_index, repeats = 1, 1
        for index in range(1, len(letters)):
            if letters[index] == letters[index-1]:
                repeats += 1
            else:
                repeats = 1
            if repeats > repeatLimit:
                try:
                    while letters[replace_index] == letters[index]:
                        replace_index += 1
                    letters[index], letters[replace_index] = letters[replace_index], letters[index]
                    replace_index += 1
                except IndexError:
                    return ''.join(letters[:index])
            if replace_index == index:
                replace_index += 1
        return ''.join(letters)

In [15]:
#1475. Final Prices With a Special Discount in a Shop
class FinalPrices:
    def finalPrices(self, prices):
        queue = deque([(prices[0], 0)])
        result = [None for _ in range(len(prices))]
        for index, price in enumerate(prices[1:], 1):
            while queue and price <= queue[-1][0]:
                cur_price, cur_index = queue.pop()
                result[cur_index] = cur_price - price 
            queue.append((price, index))
        while queue:
            cur_price, cur_index = queue.pop()
            result[cur_index] = cur_price
        return result

In [43]:
#1792. Maximum Average Pass Ratio
class MaxAverageRatio:
    def _getIncrease(self, class_info):
        num, delim = class_info
        return (delim-num) / (delim*(delim+1))
        
    def maxAverageRatio(self, classes, extraStudents):
        sorted_classes = deque(sorted(classes, key=self._getIncrease))
        for _ in range(extraStudents):
            cur_pass, cur_total = sorted_classes.pop()
            bisect.insort(sorted_classes, [cur_pass+1, cur_total+1], key=self._getIncrease)
        result = functools.reduce(lambda x,y: x+y[0]/y[1], sorted_classes, 0) / len(sorted_classes)
        return result

In [11]:
#769. Max Chunks To Make Sorted
class MaxChunksToSorted:
    def maxChunksToSorted(self, arr):
        indexes, nums = set(), set()
        result = 0
        for index, num in enumerate(arr):
            indexes.add(index)
            nums.add(num)
            if not indexes ^ nums:
                result += 1
                indexes, nums = set(), set()
        return result

In [62]:
#2762. Continuous Subarrays
class ContinuousSubarrays:
    def continuousSubarrays(self, nums):
        max_min = deque([])
        left_index = 0
        result = 0
        for right_index, num in enumerate(nums):
            bisect.insort(max_min, (num, right_index))
            if max_min[-1][0] - max_min[0][0] > 2:
                while max_min[-1][0] - max_min[0][0] > 2:
                    if max_min[-1][1] > max_min[0][1]:
                        left_index = max(left_index, max_min.popleft()[1])
                    else:
                        left_index = max(left_index, max_min.pop()[1])
                while max_min[-1][1] < left_index:
                    max_min.pop()
                while max_min[0][1] < left_index:
                    max_min.popleft()
                left_index += 1
            result += right_index - left_index + 1
        return result

In [76]:
#1346. Check If N and Its Double Exist
class CheckIfExist:
    def checkIfExist(self, arr):
        checked_nums = set()
        for num in arr:
            if (num*2 in checked_nums) or ((num%2==0) and (num/2 in checked_nums)):
                return True
            checked_nums.add(num)
        return False

In [30]:
#2415. Reverse Odd Levels of Binary Tree
class ReverseOddLevels:
    def reverseOddLevels(self, root):
        queue = deque([root])
        reversed_vals = deque([])
        lvl = 0
        while queue:
            temp_queue = deque([])
            while queue:
                cur_node = queue.popleft()
                if lvl % 2 == 1:
                    cur_node.val = reversed_vals.popleft()
                else:
                    if cur_node.right:
                        reversed_vals.appendleft(cur_node.right.val)
                        reversed_vals.appendleft(cur_node.left.val)
                if cur_node.right:
                    temp_queue.append(cur_node.right)
                    temp_queue.append(cur_node.left)
            queue = temp_queue
            lvl += 1
        return root

In [47]:
#2471. Minimum Number of Operations to Sort a Binary Tree by Level
class MinimumOperations:
    def _getValNodes(self, root, val_nodes):
        val_nodes[root.val] = root
        if root.left:
            self._getValNodes(root.left, val_nodes)
        if root.right:
            self._getValNodes(root.right, val_nodes)
        
    def minimumOperations(self, root):
        val_nodes = {}
        self._getValNodes(root, val_nodes)
        queue = deque([root])
        sorted_vals = deque([root.val])
        result = 0
        while queue:
            temp_queue = deque([])
            temp_sorted_vals = deque([])
            while queue:
                cur_node = queue.popleft()
                cur_sorted_val = sorted_vals.popleft()
                if cur_node.val != cur_sorted_val:
                    val_nodes[cur_node.val], val_nodes[cur_sorted_val] = (val_nodes[cur_sorted_val],
                                                                          val_nodes[cur_node.val])
                    val_nodes[cur_node.val].val = cur_node.val
                    cur_node.val = cur_sorted_val
                    result += 1
                if cur_node.left:
                    bisect.insort(temp_sorted_vals, cur_node.left.val)
                    temp_queue.append(cur_node.left)
                if cur_node.right:
                    bisect.insort(temp_sorted_vals, cur_node.right.val)
                    temp_queue.append(cur_node.right)
            queue = temp_queue
            sorted_vals = temp_sorted_vals
        return result

In [52]:
#3203. Find Minimum Diameter After Merging Two Trees
class MinimumDiameterAfterMerge:
    def _getTree(self, edges):
        tree = {}
        for start, end in edges:
            tree.setdefault(start, set()).add(end)
            tree.setdefault(end, set()).add(start)
        return tree

    def _getTreeDiameter(self, tree):
        result = 0
        leafs = deque(node for node in tree if len(tree[node]) == 1)
        while len(tree) > 2:
            for _ in range(len(leafs)):
                leaf = leafs.popleft()
                parent = tree[leaf].pop()
                tree.pop(leaf)
                tree[parent].remove(leaf)
                if len(tree[parent]) == 1:
                    leafs.append(parent)
            result += 1 
        return result + len(tree) - 1
        
    def minimumDiameterAfterMerge(self, edges1, edges2):
        diam_1, diam_2 = 0, 0
        roots_1, roots_2 = 0, 0
        if edges1:
            tree_1 = self._getTree(edges1)
            diam_1 = self._getTreeDiameter(tree_1)
            roots_1 += len(tree_1) - 1
        if edges2:
            tree_2 = self._getTree(edges2)
            diam_2 = self._getTreeDiameter(tree_2)
            roots_2 += len(tree_2) - 1
        return max(diam_1+diam_2+1, 2*diam_1-roots_1, 2*diam_2-roots_2)

In [74]:
#14. Longest Common Prefix
class LongestCommonPrefix:
    def longestCommonPrefix(self, strs):
        cur_index = 0
        while True:
            for string in strs:
                if ((len(string) == cur_index) 
                    or (strs[0][cur_index] != string[cur_index])):
                    return strs[0][:cur_index]
            cur_index += 1

In [25]:
#1014. Best Sightseeing Pair
class MaxScoreSightseeingPair:
    def _getPrefix(self, values):
        result = deque([])
        for index, num in list(enumerate(values))[:0:-1]:
            sum_val = num + values[0] - index
            if not result:
                result.appendleft(sum_val)
            else:
                result.appendleft(max(result[0], sum_val))
        return result
        
    def maxScoreSightseeingPair(self, values):
        prefix = self._getPrefix(values)
        result = prefix[0]
        for index, num in enumerate(values[1:-1], 1):
            diff = num - values[0] + index
            result = max(result, prefix[index]+diff)
        return result

In [20]:
#2466. Count Ways To Build Good Strings
class CountGoodStrings:
    def _getAmount(self, length_amount, cur_index, zero, one):
        result = 0
        result += length_amount[cur_index-zero] if cur_index - zero >= 0 else 0
        result += length_amount[cur_index-one] if cur_index - one >= 0 else 0
        return result
    
    def countGoodStrings(self, low, high, zero, one):
        length_amount = [1]
        while len(length_amount) <= high:
            cur_index = len(length_amount)
            length_amount.append(self._getAmount(length_amount, cur_index, zero, one))
        return sum(length_amount[low:high+1]) % (10 ** 9 + 7)

In [39]:
#983. Minimum Cost For Tickets
class MincostTickets:
    def mincostTickets(self, days, costs):
        days_total = [0] * 30
        days_set = set(days)
        for day in range(days[-1]):
            if day + 1 in days_set:
                days_total.append(min(days_total[-1]+costs[0], 
                                      days_total[-7]+costs[1],
                                      days_total[-30]+costs[2]))
            else:
                days_total.append(days_total[-1])
        return days_total[-1]

In [50]:
#1422. Maximum Score After Splitting a String
class MaxScore:
    def maxScore(self, s):
        zeros, ones = 0, 0
        for string in s:
            if string == '1':
                ones += 1
        result = 0
        for string in s[:-1]:
            if string == '0':
                zeros += 1
            else:
                ones -= 1
            result = max(result, zeros+ones)
        return result

In [5]:
#2270. Number of Ways to Split Array
class WaysToSplitArray:
    def waysToSplitArray(self, nums):
        prefix = [0]
        for num in nums:
            prefix.append(prefix[-1]+num)
        result = 0
        for index in range(1, len(prefix)-1):
            if prefix[index] >= prefix[-1] - prefix[index]:
                result += 1
        return result

In [4]:
#1769. Minimum Number of Operations to Move All Balls to Each Box
class MinOperations:
    def minOperations(self, boxes):
        left, right = 1 if boxes[0] == '1' else 0, 0
        result = [0]
        for index, box in enumerate(boxes[1:], 1):
            if box == '1':
                right += 1
                result[0] += index
        for box in boxes[1:]:
            result.append(result[-1]-right+left)
            if box == '1':
                right -= 1
                left += 1
        return result

In [18]:
#1408. String Matching in an Array
class StringMatching:
    def stringMatching(self, words):
        result = set()
        for cur_index, cur_word in enumerate(words[:-1], 1):
            for word in words[cur_index:]:
                if cur_word.find(word) >= 0:
                    result.add(word)
                if word.find(cur_word) >= 0:
                    result.add(cur_word)
        return list(result)

In [10]:
#3042. Count Prefix and Suffix Pairs I
class CountPrefixSuffixPairs:
    def isPrefixAndSuffix(self, check_string, main_string):
        return ((main_string[:len(check_string)] == check_string) 
                and (main_string[-len(check_string):] == check_string))
    
    def countPrefixSuffixPairs(self, words):
        result = 0
        for cur_index, cur_word in enumerate(words[:-1], 1):
            for word in words[cur_index:]:
                result += self.isPrefixAndSuffix(cur_word, word)
        return result